# BCI competition 2a

C:\Users\pipo_\Downloads\desc_2a

In [20]:
import mne
import numpy as np
from mne.datasets import eegbci
import matplotlib.pyplot as plt
from os import listdir
from mne.channels import make_standard_montage
from scipy import signal
from scipy.linalg import sqrtm, inv 
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.utils import shuffle
from mne.decoding import CSP
from sklearn.svm import SVC
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit,StratifiedKFold ,cross_val_score, cross_val_predict, KFold
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from pyriemann.utils.distance import distance_riemann
from scipy.linalg import logm, expm


target_data_0 = "A01"
calibrate_size = 60
alignmentMethod = "EA"

condition_wLTL= "EA"


In [21]:
def GetRawEDF(target_subjects="pipo"):
    EEG_data = {}

    if target_subjects == "all":
        target_subjects = ["A01","A02","A03","A04","A05","A06","A07","A08","A09"]

    path = "D:\\BCI_Competition_2a\\Training\\"
    list_dir = listdir(path)

    for i in range (0,len(target_subjects)):

        raw_gdf = mne.io.read_raw_gdf(path+list_dir[i],preload = False, verbose = False)

        EEG_data[target_subjects[i]] = {"Raw_data": raw_gdf.copy()}

    print(f"Successful to create Data of {target_subjects}")

    return EEG_data

In [22]:
Raw_data = GetRawEDF(target_subjects="all")

c:\Python311\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)
c:\Python311\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)
c:\Python311\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)
c:\Python311\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)
c:\Python311\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)
c:\Python311\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for dup

Successful to create Data of ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09']


In [23]:
def butter_bandpass(lowcut,highcut,fs,order):
    nyq = 0.5*fs
    low = lowcut/nyq
    high = highcut/nyq
    b,a = signal.butter(order,[low,high],'bandpass')
    return b,a

def butter_bandpass_filter(data,lowcut = 6,highcut = 30, order = 4):
    b,a = butter_bandpass(lowcut,highcut,250,order)
    y = signal.filtfilt(b,a,data,axis=2)
    return y

def GetEpoch(EEG_data, tmin=-2.0, tmax=6.0, crop=(0,2),baseline = (-0.5,0.0), trial_removal_th = 100):

    EEG_epoch = {}

    for key_subs in EEG_data:
        raw_edf = EEG_data[key_subs]["Raw_data"]

        events, event_dict = mne.events_from_annotations(raw_edf)

        if key_subs == 'A04':
            event_dict =  {'769': 5,
            '770': 6,
            '772': 8,
            '771': 7}
            mapping = {5: 0, 6: 1, 8: 2, 7: 3}
            selected_events = events[np.isin(events[:, 2], [5, 6, 7, 8])]

        else:
            event_dict =  {'769': 7,
            '770': 8,
            '772': 10,
            '771': 9}
            mapping = {7: 0, 8: 1, 10: 2, 9: 3}
            selected_events = events[np.isin(events[:, 2], [7, 8, 9, 10])]

        Epochs = mne.Epochs(raw_edf, selected_events, 
            tmin= tmin,  
            tmax= tmax,    
            event_id=event_dict,
            preload = True,
            event_repeated='drop',
            baseline=baseline,
            verbose=False
            )
        
        selected_ch = ['EEG-Fz', 'EEG-Cz', 'EEG-C3', 'EEG-C4', 'EEG-Pz']
        
        EEG_epoch[key_subs] =  {"Raw_Epoch": Epochs.copy().pick(selected_ch).crop(tmin= crop[0], tmax= crop[1])}

        train_data = EEG_epoch[key_subs]['Raw_Epoch'].copy().get_data() * 10e5 

        labels = EEG_epoch[key_subs]["Raw_Epoch"].copy().events[:,-1]

        labels = np.vectorize(mapping.get)(labels)

        outlier_trial = []
        for ii in range(0,train_data.shape[0]):
            if train_data[ii].max() > trial_removal_th or train_data[ii].min() < -trial_removal_th:
                outlier_trial.append(ii)
                print(key_subs,train_data[ii].min(), ii)
                print(key_subs,train_data[ii].max(), ii)

        EEG_epoch[key_subs]['Raw_Epoch'] = np.delete(train_data, outlier_trial, axis = 0)
        EEG_epoch[key_subs]['label'] = np.delete(labels, outlier_trial)

        filtered_data = butter_bandpass_filter(EEG_epoch[key_subs]['Raw_Epoch'], lowcut= 6, highcut= 32)
        EEG_epoch[key_subs]['Raw_Epoch'] = filtered_data

    return EEG_epoch

In [24]:
EEG_Epochs = GetEpoch(Raw_data ,tmin= -1.0, tmax= 4.0, crop = (0,4) ,baseline= (-0.5,0.0), trial_removal_th = 150)

Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']


C:\Users\pipo_\AppData\Local\Temp\ipykernel_13236\1188733013.py:52: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].copy().get_data() * 10e5


Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']


C:\Users\pipo_\AppData\Local\Temp\ipykernel_13236\1188733013.py:52: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].copy().get_data() * 10e5


Used Annotations descriptions: ['1023', '1072', '32766', '768', '769', '770', '771', '772']


...

C:\Users\pipo_\AppData\Local\Temp\ipykernel_13236\1188733013.py:52: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].copy().get_data() * 10e5


Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']


C:\Users\pipo_\AppData\Local\Temp\ipykernel_13236\1188733013.py:52: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].copy().get_data() * 10e5


Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']


C:\Users\pipo_\AppData\Local\Temp\ipykernel_13236\1188733013.py:52: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].copy().get_data() * 10e5
C:\Users\pipo_\AppData\Local\Temp\ipykernel_13236\1188733013.py:52: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].copy().get_data() * 10e5


In [25]:
def GetConfusionMatrix(models, X_train, X_test, y_train, y_test):
    y_pred = models.predict(X_train)
    print("Classification TRAIN DATA \n=======================")
    print(classification_report(y_true= y_train, y_pred=y_pred))
    print("Confusion matrix \n=======================")
    print(confusion_matrix(y_true= y_train, y_pred=y_pred))

    y_pred = models.predict(X_test)
    print("Classification TEST DATA \n=======================")
    print(classification_report(y_true=y_test, y_pred=y_pred))
    print("Confusion matrix \n=======================")
    print(confusion_matrix(y_true=y_test, y_pred=y_pred))
    

label_target = EEG_Epochs[target_data_0]['label']
x_train, x_test, y_train, y_test = train_test_split(EEG_Epochs[target_data_0]['Raw_Epoch'], label_target, test_size=0.3, random_state = 42, stratify=label_target)

csp = CSP(n_components = 5, reg=None, log=None, rank= 'info')
csp.fit(x_train, y_train)   

x_train = csp.transform(x_train)
x_test = csp.transform(x_test)

lda = LinearDiscriminantAnalysis()
score = cross_val_score(lda, x_train, y_train, cv= 10)
print("LDA only Cross-validation scores:", np.mean(score))
lda.fit(x_train, y_train)

GetConfusionMatrix(lda, x_train, x_test, y_train, y_test)


Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL


Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
LDA only Cross-validation scores: 0.6516666666666666
Classification TRAIN DATA 
              precision    recall  f1-score   support

           0       0.72      0.66      0.69        50
           1       0.69      0.75      0.72        51
           2       0.72      0.78      0.75        50
           3       0.65      0.60      0.63        50

    accuracy                           0.70       201
   macro avg       0.70      0.70      0.69       201
weighted avg

In [26]:
from BCIAllFunction import BCIFuntions

calibrate_size = calibrate_size / EEG_Epochs[target_data_0]['Raw_Epoch'].shape[0]
AllBCIClass = BCIFuntions(numclass = 4, frequency = 250, ch_pick = ['Fz','C3', 'Cz','C4','Pz'])


if alignmentMethod == "LA":
    AllBCIClass.ComputeLA(EEG_Epochs, target_subject= target_data_0, calibrate_size=calibrate_size)
    if calibrate_size != 0:
        target_data = target_data_0 + "_test"
    count = 0

    for index in range(len(EEG_Epochs[target_data]['KMediod_label'])):
        if EEG_Epochs[target_data]['label'][index] == EEG_Epochs[target_data]['KMediod_label'][index]:
            count += 1
    print(count/len(EEG_Epochs[target_data]['KMediod_label']) * 100)
    
else:
    AllBCIClass.GetRawSet_ComputeEA(EEG_Epochs, target_subject= target_data_0, calibrate_size=calibrate_size)
    if calibrate_size != 0:
        target_data = target_data_0 + "_test"

In [27]:
EEG_Epochs['A01'].keys()

dict_keys(['Raw_Epoch', 'label', 'Raw_left', 'Raw_right', 'Raw_non', 'Raw_feet', 'EA_left', 'EA_right', 'EA_feet', 'EA_non', 'EA_Epoch'])

In [28]:
AllBCIClass.classifyCSP_LDA(EEG_Epochs, target_subjects= target_data, calibrate_data= target_data_0, condition = "noEA") #target_sub is used for target_data otherwise are source_data

Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
LDA only Cross-validation scores: 0.3815377376246941
Classification TRAIN DATA 
              precision    recall  f1-score   support

           0       0.36      0.42      0.39       576
           1       0.41      0.50      0.45       576
           2       0.41      0.36      0.38       576
       

In [29]:
AllBCIClass.classifyCSP_LDA(EEG_Epochs, target_subjects= target_data, calibrate_data= target_data_0,condition = "EA") 

Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
LDA only Cross-validation scores: 0.4674948240165631
Classification TRAIN DATA 
              precision    recall  f1-score   support

           0       0.49      0.53      0.51       576
           1       0.49      0.50      0.49       576
           2       0.47      0.51      0.49       576
       

# WLTL

In [30]:
CSP2D_Epoch = AllBCIClass.computeCSPFeatures(EEG_Epochs, target_subject = target_data)

Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5


In [31]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

# Custom loss function
class CustomLossLL1(tf.keras.losses.Loss):
    def __init__(self, lambda_t, model):
        super().__init__()
        self.lambda_t = lambda_t
        self.cross_entropy = CategoricalCrossentropy()
        self.model = model

    def call(self, y_true, y_pred):
        ce_loss = self.cross_entropy(y_true, y_pred)
        ws = self.get_weights_from_model()
        reg_term = self.regularization_term(ws)
        return ce_loss + self.lambda_t * reg_term

    def get_weights_from_model(self):
        model_weights = []
        for layer in self.model.layers:
            if len(layer.get_weights()) > 0:
                model_weights.append(layer.get_weights()[0])
        # return tf.concat([tf.reshape(w, [-1]) for w in model_weights], axis=0)
        return model_weights

    def regularization_term(self, ws):
        reg_term = tf.pow(tf.norm(ws, ord='euclidean'),2)
        return reg_term


# Custom training loop
def custom_train_step(model, optimizer, x, y, custom_loss):
    with tf.GradientTape() as tape:
        y_pred = model(x, training=True) # Perform a forward pass and compute the predictions
        loss = custom_loss(y, y_pred) # Compute the custom loss
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


def train_weight_LL(X_train, y_train, lambd, num_tier=10, learning_rate = 0.01):
    n_classes = np.unique(y_train).size
    y_one_hot = tf.keras.utils.to_categorical(y_train, num_classes=n_classes)

    lambda_t = lambd  # Regularization parameter

    model = Sequential([
        Dense(n_classes, input_shape=(X_train.shape[1],), activation='softmax')  # Adjust input_shape to match the number of features in X
    ])

    # Compile the model
    optimizer = Adam(learning_rate)
    custom_loss = CustomLossLL1(lambda_t, model)

    # Custom training loop
    epochs = num_tier
    lowest_loss = float('inf')
    best_weights = None
    for epoch in range(epochs):
        loss = custom_train_step(model, optimizer, X_train, y_one_hot, custom_loss)
        loss_value = loss.numpy()
        if loss_value < lowest_loss:
            lowest_loss = loss_value
            best_weights = [layer.get_weights() for layer in model.layers]

        if epoch % 20 == 0:
            print(f"Epoch {epoch}, Loss: {loss.numpy()}")

    # best_weights = [layer.get_weights() for layer in model.layers]

    return best_weights, lowest_loss

def build_clf_params(data, target_subjects ,condition):

    for sub in data.keys():
        if (sub  != target_subjects) and  (sub != target_data_0): #Don't apply weight to target subject
            # Where the tranining data is stored
            if condition == "noEA":
                X = data[sub]['Raw_csp']
                y = data[sub]['Raw_csp_label']
                store_ws = 'ws_Raw'

            else:
                X = data[sub]['EA_csp']
                y = data[sub]['EA_csp_label']
                store_ws = 'ws_EA'

            weights, loss = train_weight_LL(X_train=X, y_train=y, lambd= 0.1, num_tier=1000, learning_rate= 0.005)
            print("weights of ", str(sub), ": ", weights)
            print("Lowest loss of ", str(sub), ": ", loss)
            data[sub][store_ws] = weights

build_clf_params(CSP2D_Epoch, target_subjects= target_data ,condition = condition_wLTL)

Epoch 0, Loss: 1.6863042116165161
Epoch 20, Loss: 1.6186344623565674
Epoch 40, Loss: 1.5694645643234253
Epoch 60, Loss: 1.540996789932251
Epoch 80, Loss: 1.53383469581604
Epoch 100, Loss: 1.546771764755249
Epoch 120, Loss: 1.5777798891067505
Epoch 140, Loss: 1.624530553817749
Epoch 160, Loss: 1.6846745014190674
Epoch 180, Loss: 1.755986213684082
Epoch 200, Loss: 1.8364484310150146
Epoch 220, Loss: 1.924293041229248
Epoch 240, Loss: 2.018007516860962
Epoch 260, Loss: 2.116323709487915
Epoch 280, Loss: 2.2181975841522217
Epoch 300, Loss: 2.3227806091308594
Epoch 320, Loss: 2.4293899536132812
Epoch 340, Loss: 2.5374841690063477
Epoch 360, Loss: 2.646635055541992
Epoch 380, Loss: 2.7565059661865234
Epoch 400, Loss: 2.8668341636657715
Epoch 420, Loss: 2.977416753768921
Epoch 440, Loss: 3.088094711303711
Epoch 460, Loss: 3.198747158050537
Epoch 480, Loss: 3.3092832565307617
Epoch 500, Loss: 3.4196338653564453
Epoch 520, Loss: 3.529752254486084
Epoch 540, Loss: 3.6396048069000244
Epoch 560, L

In [32]:
# First define the kl divergence
def KL_div(P, Q):
    # First convert to np array
    P = np.array(P)
    Q = np.array(Q)
    
    # Then compute their means, datain shape of samples x feat
    mu_P = np.mean(P, axis=0)
    mu_Q = np.mean(Q, axis=0)    

    
    # Compute their covariance
    sigma_P = np.cov(P, rowvar=False)
    sigma_Q = np.cov(Q, rowvar=False)  

    diff = mu_Q - mu_P

    inv_sigma_Q = np.linalg.inv(sigma_Q)
    term1 = np.dot(np.dot(diff.T, inv_sigma_Q), diff)
    
    # Calculate the trace term trace(Sigma_Q^{-1} * Sigma_P)
    term2 = np.trace(np.dot(inv_sigma_Q, sigma_P))
    
    # Calculate the determinant term ln(det(Sigma_P) / det(Sigma_Q))
    det_sigma0 = np.linalg.det(sigma_P)
    det_sigma1 = np.linalg.det(sigma_Q)

    
    epsilon = 1e-6
    term3 = np.log((det_sigma0+epsilon) / (det_sigma1+epsilon))
    
    print(term3)
    
    # Dimensionality of the data
    K = mu_P.shape[0]
    
    # KL divergence
    kl_div = 0.5 * (term1 + term2 - term3 - K)
    
    return kl_div

In [33]:
# Compute kl divergence of target subject to each source subject
def compute_all_kl_div(data, target_subjects , condition):
    '''
    Parameter:
    data, is the whole data containing target and source data
    '''
    kl_div_score = []

    if condition == "noEA":
        target_data = 'Raw_csp'
        label_name = 'Raw_csp_label'

    else:
        target_data = 'EA_csp'
        label_name = 'EA_csp_label'
        
    # cal P from target data
    label_tgt =  data[target_subjects][label_name]
    P_left =  data[target_subjects][target_data][np.where(label_tgt == 0)]
    P_right = data[target_subjects][target_data][np.where(label_tgt == 1)]
    P_non = data[target_subjects][target_data][np.where(label_tgt == 2)]
    P_feet = data[target_subjects][target_data][np.where(label_tgt == 3)]

    tgt_data = target_subjects + "_test"

    #cal Q from each source subject
    for sub in data.keys():
        if (sub != target_subjects) and (sub != tgt_data):
            label_src =  data[sub][label_name]
            Q_left =  data[sub][target_data][np.where(label_src == 0)]
            Q_right = data[sub][target_data][np.where(label_src == 1)]
            Q_non = data[sub][target_data][np.where(label_src == 2)]
            Q_feet = data[sub][target_data][np.where(label_src == 3)]

            kl_left = KL_div(P_left, Q_left)
            kl_right = KL_div(P_right, Q_right)
            kl_non = KL_div(P_non, Q_non)
            kl_feet = KL_div(P_feet, Q_feet)

            # kl_div = (kl_left + kl_right+ kl_non + kl_feet)/4

            kl_div_temp = [kl_left, kl_right, kl_non, kl_feet]

            kl_div_score.append(kl_div_temp)

    data[target_subjects]['kl_div'] = kl_div_score


compute_all_kl_div(CSP2D_Epoch, target_subjects=target_data_0 ,condition = condition_wLTL) #target_sub for cal KL is calibrate set

-0.013046335809121101
-0.020345206185433222
-0.03704890610853611
-0.0036543318428413907
-0.11547122673100808
-0.12188058181832209
-0.08019869394593408
-0.15258691487612716
-0.009117815331594123
-0.03195924974387014
-0.019888838834848314
-0.03363407838011897
-0.07766147716213788
-0.1339580591356929
-0.10716656578592094
-0.2236419578727472
-0.792516492604584
-0.38985354537227546
-0.3780520634350904
-0.5243189236006929
0.009963885156666026
-0.020956145697663475
-0.020027596217610966
-0.010764495085184646
-0.06390564747268009
-0.0371600264329485
-0.08077822045638713
-0.07884308942420591
-0.8778010591307759
-0.5122005605274579
-0.15656597153216567
-0.6767878638826109


In [34]:
np.array(CSP2D_Epoch[target_data_0]['kl_div'])

array([[ 4.42320557,  2.08335446,  5.78887027,  1.66003968],
       [ 1.23518178,  0.45930739,  0.35049121, -0.64477476],
       [ 3.18623324,  1.07699435,  2.15712201, -0.08035444],
       [ 2.3720294 ,  1.27986848,  3.56701029, -0.85198609],
       [ 0.10415563,  0.26472214,  1.08903794, -0.85246588],
       [ 5.00787935,  1.23511343,  2.01016668,  1.21894158],
       [ 3.66177753,  0.78463994,  0.87388955, -0.17226742],
       [-0.52804668, -0.10113967,  0.02055188, -0.70714614]])

In [35]:
def compute_similarity_weights(data, target_subjects):
    kl = data[target_subjects]['kl_div']
    KL_inv_left = []
    KL_inv_right = []
    KL_inv_non = []
    KL_inv_feet = []

    alpha_s = []
    eps = 0.0001
    
    #equation (9)
    for val in kl:
        if val != 0: 
            KL_inv_left.append(1/((val[0] + eps)**4))
            KL_inv_right.append(1/((val[1] + eps)**4))
            KL_inv_non.append(1/((val[2] + eps)**4))
            KL_inv_feet.append(1/((val[3] + eps)**4))

    print(KL_inv_left)
    print(KL_inv_right)
    
    for i in range(0,len(KL_inv_left)):
        temp = [KL_inv_left[i]/sum(KL_inv_left), KL_inv_right[i]/sum(KL_inv_right), KL_inv_non[i]/sum(KL_inv_non), KL_inv_feet[i]/sum(KL_inv_feet)]
        alpha_s.append(temp)

    print(np.array(alpha_s))
                
    data[target_subjects]['alpha_s'] = alpha_s

compute_similarity_weights(CSP2D_Epoch, target_subjects=target_data_0)

[0.0026122349312512195, 0.4294729870447311, 0.009701418086893959, 0.03158249952373265, 8464.510828289724, 0.0015898270541893347, 0.005561407760538303, 12.871813705717567]
[0.05307181644730946, 22.449573326988762, 0.7429934991796989, 0.3725657280700145, 203.32128831847328, 0.4295680566394077, 2.636920109705572, 9594.719183163676]
[[3.08124215e-07 5.40186270e-06 1.61967392e-10 5.21217005e-06]
 [5.06581645e-05 2.28500777e-03 1.20400634e-05 2.29210458e-04]
 [1.14432350e-06 7.56248635e-05 8.39950113e-09 9.54378837e-01]
 [3.72529008e-06 3.79212367e-05 1.12348355e-09 7.51745784e-05]
 [9.98425035e-01 2.06948576e-02 1.29271061e-07 7.50054593e-05]
 [1.87526860e-07 4.37231627e-05 1.11382809e-08 1.79276362e-05]
 [6.55991687e-07 2.68396323e-04 3.11751035e-07 4.50602138e-02]
 [1.51828515e-03 9.76589067e-01 9.99987498e-01 1.58418565e-04]]


In [36]:
def compute_ETL_and_mu_ws(data, target_subjects, condition):

    mu_ws = 0
    temp_ws = 0

    if condition == "noEA":
        ws_name = 'ws_Raw'
    else:
        ws_name = 'ws_EA'

    alpha_s = np.array(data[target_subjects]['alpha_s'])

    tgt_data = target_subjects + "_test"
    index_count = 0

    for sub in data.keys():
        if (sub != target_subjects) and (sub != tgt_data):
            ws = data[sub][ws_name][0][0]
            # mu_ws += ws @ alpha_s  #equation (10)
            # mu_ws += np.dot(ws, np.transpose(alpha_s))
            mu_ws += ws * alpha_s[index_count]
            index_count += 1

    print(np.array(mu_ws))

    index_count = 0
    for sub in data.keys():
        if (sub != target_subjects) and (sub != tgt_data):
            ws = data[sub][ws_name][0][0]
            # ws_min_mu = np.dot((np.dot(ws,np.transpose(alpha_s)) - mu_ws) , np.transpose((np.dot(ws,np.transpose(alpha_s)) - mu_ws)))
            ws_min_mu = np.dot(((ws * alpha_s[index_count]) - mu_ws), np.transpose((ws * alpha_s[index_count]) - mu_ws))
            temp_ws += ws_min_mu #equation (11)
            index_count += 1

    print(np.array(temp_ws))
    
    # den = np.diag(temp_ws) #get array in diagonal line

    den = temp_ws
    nom = np.trace(temp_ws) #Return the sum along diagonals of the array.
    Sigma_TL = den/nom


    data[target_subjects]['Sigma_TL'] = Sigma_TL
    data[target_subjects]['mu_ws'] = mu_ws

compute_ETL_and_mu_ws(CSP2D_Epoch, target_subjects = target_data_0, condition=condition_wLTL)

[[ 0.25269806  0.17138827 -0.36860698 -0.47900679]
 [ 0.49221731 -0.1275219   0.67049592 -0.03716568]
 [ 0.12493296 -0.03578234 -0.24071086  0.53747992]
 [ 0.61984348 -0.44546507  0.6215602   0.02699506]
 [-0.06913421 -0.03574355 -0.13218919 -0.24457191]]
[[ 3.22115305 -0.88961543 -1.02450094 -1.13448107  1.00520276]
 [-0.88961543  4.96680777 -0.80309866  5.44433701 -0.76217483]
 [-1.02450094 -0.80309866  2.57491708 -0.2796031  -0.76348048]
 [-1.13448107  5.44433701 -0.2796031   6.78515259 -0.80725858]
 [ 1.00520276 -0.76217483 -0.76348048 -0.80725858  0.59384656]]


In [37]:
print(np.array(CSP2D_Epoch[target_data_0]['Sigma_TL']).shape)
print(np.array(CSP2D_Epoch[target_data_0]['mu_ws']).shape)

(5, 5)
(5, 4)


In [38]:
# Custom loss function
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

class CustomLossLL2(tf.keras.losses.Loss):
    def __init__(self, lambda_t, model, mu, sigma_TL):
        super().__init__()
        self.lambda_t = lambda_t
        self.cross_entropy = CategoricalCrossentropy()
        self.model = model
        self.mu = tf.convert_to_tensor(mu, dtype=tf.float32)
        self.sigma_TL = tf.convert_to_tensor(sigma_TL, dtype=tf.float32)

    def call(self, y_true, y_pred):
        ce_loss = self.cross_entropy(y_true, y_pred)
        wt = self.get_weights_from_model()
        reg_term = self.regularization_term(wt)

        return ce_loss + (self.lambda_t * tf.linalg.matmul(reg_term, wt))

    def get_weights_from_model(self):
        model_weights = []
        for layer in self.model.layers:
            if len(layer.get_weights()) > 0:
                model_weights.append(layer.get_weights()[0])
        # return tf.concat([tf.reshape(w, [-1]) for w in model_weights], axis=0)
        return model_weights

    def regularization_term(self, wt):
        diff = wt - self.mu
        reg_term = 0.5 * tf.linalg.matmul(tf.linalg.matmul(tf.linalg.inv(self.sigma_TL), diff[0]), tf.transpose(diff[0]))
        reg_term += 0.5 * tf.math.log(tf.linalg.det(self.sigma_TL))
        return reg_term


# Custom training loop
def custom_train_step(model, optimizer, x, y, custom_loss):
    with tf.GradientTape() as tape:
        y_pred = model(x, training=True) # Perform a forward pass and compute the predictions
        loss = custom_loss(y, y_pred) # Compute the custom loss
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


def train_weight_LL2(X_train, y_train, lambd, mu, sigma_TL, num_tier=10, learning_rate = 0.01):
    n_classes = np.unique(y_train).size
    y_one_hot = tf.keras.utils.to_categorical(y_train, num_classes=n_classes)

    lambda_t = lambd  # Regularization parameter

    model = Sequential([
        Dense(n_classes, input_shape=(X_train.shape[1],), activation='softmax')  # Adjust input_shape to match the number of features in X
    ])

    # Compile the model
    optimizer = Adam(learning_rate)
    custom_loss = CustomLossLL2(lambda_t, model, mu, sigma_TL)

    # Custom training loop
    epochs = num_tier
    lowest_loss = float('inf')
    best_weights = None
    best_model = None

    for epoch in range(epochs):
        loss = custom_train_step(model, optimizer, X_train, y_one_hot, custom_loss)
        loss_value = loss.numpy()
        if epoch % 20 == 0:
            print(f"Epoch {epoch}, Loss: {loss.numpy()}")

        if (abs(loss_value) < lowest_loss):
            lowest_loss = abs(loss_value)
            best_model = model
            best_weights = [layer.get_weights() for layer in model.layers]

    return best_model, best_weights, lowest_loss

def GetConfusionMatrix(model, X_train, X_test, y_train, y_test):
    y_pred_prob = model.predict(X_train)
    y_pred = np.argmax(y_pred_prob, axis=1)

    print("Classification TRAIN DATA \n=======================")
    print(classification_report(y_true= y_train, y_pred=y_pred))
    print("Confusion matrix \n=======================")
    print(confusion_matrix(y_true= y_train, y_pred=y_pred))

    y_pred_prob = model.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)
    print("Classification TEST DATA \n=======================")
    print(classification_report(y_true=y_test, y_pred=y_pred))
    print("Confusion matrix \n=======================")
    print(confusion_matrix(y_true=y_test, y_pred=y_pred))


def tgt_test_wLTL(data, target_subjects ,condition):
        tgt_data = target_subjects + "_test"

        if condition == "noEA":
            X = data[target_subjects]['Raw_csp']
            y = data[target_subjects]['Raw_csp_label']
            X_test = data[tgt_data]['Raw_csp']
            y_test = data[tgt_data]['Raw_csp_label']
            store_ws = 'wt_Raw'

        else:
            X = data[target_subjects]['EA_csp']
            y = data[target_subjects]['EA_csp_label']
            X_test = data[tgt_data]['EA_csp']
            y_test = data[tgt_data]['EA_csp_label']
            store_ws = 'wt_EA'

        mu = data[target_subjects]['mu_ws']
        sigma_TL = data[target_subjects]['Sigma_TL']

        X_train = X
        y_train = y
        
        model, weights, loss = train_weight_LL2(X_train=X_train, y_train=y_train, mu =mu, sigma_TL=sigma_TL, lambd= 0.1, num_tier=5000, learning_rate= 0.01)
        print("weights of ", str(target_subjects), ": ", weights)
        print("loss of ", str(target_subjects), ": ", loss)
        data[target_subjects][store_ws] = weights

        GetConfusionMatrix(model, X_train, X_test, y_train, y_test)

tgt_test_wLTL(CSP2D_Epoch, target_subjects= target_data_0 ,condition = condition_wLTL)

Epoch 0, Loss: 87.7649154663086
Epoch 20, Loss: 117.83604431152344
Epoch 40, Loss: 107.62225341796875
Epoch 60, Loss: 102.5091552734375
Epoch 80, Loss: 101.28036499023438
Epoch 100, Loss: 103.3765869140625
Epoch 120, Loss: 106.49283599853516
Epoch 140, Loss: 110.0731201171875
Epoch 160, Loss: 113.44339752197266
Epoch 180, Loss: 116.2527847290039
Epoch 200, Loss: 118.3266372680664
Epoch 220, Loss: 119.55570983886719
Epoch 240, Loss: 119.89937591552734
Epoch 260, Loss: 119.34516906738281
Epoch 280, Loss: 117.894775390625
Epoch 300, Loss: 115.55709075927734
Epoch 320, Loss: 112.3413314819336
Epoch 340, Loss: 108.2550048828125
Epoch 360, Loss: 103.3022689819336
Epoch 380, Loss: 97.48307800292969
Epoch 400, Loss: 90.7938232421875
Epoch 420, Loss: 83.2267837524414
Epoch 440, Loss: 74.77128601074219
Epoch 460, Loss: 65.4132080078125
Epoch 480, Loss: 55.1358642578125
Epoch 500, Loss: 43.92036056518555
Epoch 520, Loss: 31.745458602905273
Epoch 540, Loss: 18.588281631469727
Epoch 560, Loss: 4.42